In [1]:
import numpy as np 
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df=df.dropna()

In [4]:
df.reset_index(inplace=True)

In [5]:
#### Get the Independent Features

X=df.drop('label',axis=1)


In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 5)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf
tf.__version__


'2.1.0'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential  
from tensorflow.keras.preprocessing.text import one_hot #converting a sent into one hot given a vocab size
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
### Vocabulary size
voc_size=5000

## Onehot Representation

In [13]:
message=X.copy()

In [14]:
import nltk 
import re
from nltk.corpus import stopwords

In [16]:
#Dataset Preprocessing 
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

In [18]:
for i in range(0, (len(message))):
    review=re.sub('[^a-zA-Z]',' ',message['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(words) for words in review if not words in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [19]:
corpus[5]

'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart'

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr   #representing index of words in a particular sentence based on vocsize 

[[4427, 4992, 288, 3234, 3458, 902, 4699, 1540, 1636, 768],
 [1767, 3341, 2024, 4330, 2023, 366, 936],
 [3937, 1571, 3134, 813],
 [9, 4157, 4569, 4361, 3032, 4903],
 [4740, 2023, 344, 1201, 1600, 2754, 2023, 2038, 4234, 4571],
 [3469,
  1969,
  764,
  353,
  2961,
  3923,
  3921,
  3258,
  3893,
  2119,
  3009,
  3204,
  27,
  3250,
  936],
 [1502, 3018, 2494, 2371, 3140, 755, 3213, 1984, 4794, 4281, 3055],
 [1750, 4912, 142, 1240, 3960, 3173, 3923, 3203, 4794, 4281, 3055],
 [2694, 921, 3405, 1122, 2501, 4556, 172, 2267, 3923, 1341],
 [4580, 2214, 1892, 4729, 3744, 3481, 3850, 3488],
 [3675, 2608, 2379, 3277, 206, 1612, 1425, 4985, 3590, 3241, 2764],
 [4361, 3385, 3458, 4556, 3923, 3960],
 [325, 1109, 3458, 3847, 1072, 2413, 4487, 1164, 2030],
 [149, 2684, 899, 1019, 2654, 4093, 1874, 4794, 4281, 3055],
 [1713, 4017, 2608, 2942, 2716, 4794, 4281, 3055],
 [811, 1740, 3297, 381, 833, 1817, 2678, 848, 1209, 92],
 [3236, 4907, 3341],
 [2242, 1978, 2129, 4432, 3923, 658, 1681, 936],
 [1251,

## Embedding Representation

In [23]:
sent_length=20   #to make the sentences of fixed length
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1540 1636  768]
 [   0    0    0 ... 2023  366  936]
 [   0    0    0 ... 1571 3134  813]
 ...
 [   0    0    0 ... 4794 4281 3055]
 [   0    0    0 ... 4556  517 3651]
 [   0    0    0 ... 1626 2885  711]]


In [24]:
len(embedded_docs)

18285

In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4427,
       4992,  288, 3234, 3458,  902, 4699, 1540, 1636,  768])

In [26]:
## Creating Model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))   #1 LSTM layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
len(embedded_docs),y.shape

(18285, (18285,))

In [28]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [31]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 22s 2ms/sample - loss: 0.3343 - accuracy: 0.8402 - val_loss: 0.1962 - val_accuracy: 0.9183
Epoch 2/10
12250/12250 [==============================] - 10s 852us/sample - loss: 0.1354 - accuracy: 0.9453 - val_loss: 0.1888 - val_accuracy: 0.9231
Epoch 3/10
12250/12250 [==============================] - 10s 833us/sample - loss: 0.0886 - accuracy: 0.9673 - val_loss: 0.2361 - val_accuracy: 0.9211
Epoch 4/10
12250/12250 [==============================] - 10s 824us/sample - loss: 0.0557 - accuracy: 0.9802 - val_loss: 0.2971 - val_accuracy: 0.9201
Epoch 5/10
12250/12250 [==============================] - 10s 830us/sample - loss: 0.0285 - accuracy: 0.9902 - val_loss: 0.3450 - val_accuracy: 0.9162
Epoch 6/10
12250/12250 [==============================] - 10s 827us/sample - loss: 0.0177 - accuracy: 0.9949 - val_loss: 0.3741 - val_accuracy: 0.9176
Epoch 7/10
12250/12250 [=======================

## Adding Dropout

In [41]:
#from tensorflow.keras.layers import Dropout
## Creating model
#embedding_vector_features=40
#model=Sequential()
#model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
#model.add(Dropout(0.3))
#model.add(LSTM(100))
#model.add(Dropout(0.3))
#model.add(Dense(1,activation='sigmoid'))
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## Performance Metrics And Accuracy

In [42]:
y_pred=model.predict_classes(X_test)

In [43]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1767, 1652],
       [ 728, 1888]], dtype=int64)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6056338028169014